# EOA Feature Extraction

In [4]:
data = """
{
    "status": "1",
    "message": "OK",
    "result": [
        {
            "blockNumber": "6127109",
            "timeStamp": "1533974291",
            "hash": "0x1890d018b54fc773ca153701f64b0668d278e15ee9f99abad11635d24ec0babe",
            "nonce": "0",
            "blockHash": "0xa4a5635e484879021678290a785d8ef245c959d6f1613e9ec0f94ce13c088c8c",
            "transactionIndex": "105",
            "from": "0x8ddfdf60aaffe05c623ba193a186abd1f8024946",
            "to": "0xbceaa0040764009fdcff407e82ad1f06465fd2c4",
            "value": "25533614328758401081460",
            "gas": "21000",
            "gasPrice": "9000000000",
            "isError": "0",
            "txreceipt_status": "1",
            "input": "0x",
            "contractAddress": "",
            "cumulativeGasUsed": "7124989",
            "gasUsed": "21000",
            "confirmations": "7714599"
        },
        {
            "blockNumber": "7357680",
            "timeStamp": "1552439119",
            "hash": "0x2fce3b3cdbf88e70ecc82a334e63459f6cbe0a1dff83c0c506f20358318a23ac",
            "nonce": "0",
            "blockHash": "0x6732d7cba24e9d1af9e7491ff63e6fbfb717059e277f71f700f1abb6fb5ef032",
            "transactionIndex": "82",
            "from": "0xbceaa0040764009fdcff407e82ad1f06465fd2c4",
            "to": "0xf27b6923ed24eed02de7686962339db00a52d2aa",
            "value": "100000000000000000000",
            "gas": "21000",
            "gasPrice": "10000000000",
            "isError": "0",
            "txreceipt_status": "1",
            "input": "0x",
            "contractAddress": "",
            "cumulativeGasUsed": "5379659",
            "gasUsed": "21000",
            "confirmations": "6484028"
        },
        {
            "blockNumber": "7357690",
            "timeStamp": "1552439269",
            "hash": "0x492650541da5a2839e3314885fae541e68932a5410eb708f843a250e2863d57d",
            "nonce": "1",
            "blockHash": "0xa66571dddef8080e4603355f48db5020e168a66a0377f321b7c7b9a04cb02213",
            "transactionIndex": "26",
            "from": "0xbceaa0040764009fdcff407e82ad1f06465fd2c4",
            "to": "0xf27b6923ed24eed02de7686962339db00a52d2aa",
            "value": "25433613908758401081460",
            "gas": "21000",
            "gasPrice": "10000000000",
            "isError": "0",
            "txreceipt_status": "1",
            "input": "0x",
            "contractAddress": "",
            "cumulativeGasUsed": "1769870",
            "gasUsed": "21000",
            "confirmations": "6484018"
        }
    ]
}
"""

In [6]:
import json
input_dict = json.loads(data)
input_dict

{'status': '1',
 'message': 'OK',
 'result': [{'blockNumber': '6127109',
   'timeStamp': '1533974291',
   'hash': '0x1890d018b54fc773ca153701f64b0668d278e15ee9f99abad11635d24ec0babe',
   'nonce': '0',
   'blockHash': '0xa4a5635e484879021678290a785d8ef245c959d6f1613e9ec0f94ce13c088c8c',
   'transactionIndex': '105',
   'from': '0x8ddfdf60aaffe05c623ba193a186abd1f8024946',
   'to': '0xbceaa0040764009fdcff407e82ad1f06465fd2c4',
   'value': '25533614328758401081460',
   'gas': '21000',
   'gasPrice': '9000000000',
   'isError': '0',
   'txreceipt_status': '1',
   'input': '0x',
   'contractAddress': '',
   'cumulativeGasUsed': '7124989',
   'gasUsed': '21000',
   'confirmations': '7714599'},
  {'blockNumber': '7357680',
   'timeStamp': '1552439119',
   'hash': '0x2fce3b3cdbf88e70ecc82a334e63459f6cbe0a1dff83c0c506f20358318a23ac',
   'nonce': '0',
   'blockHash': '0x6732d7cba24e9d1af9e7491ff63e6fbfb717059e277f71f700f1abb6fb5ef032',
   'transactionIndex': '82',
   'from': '0xbceaa0040764009fd

In [178]:
class EOA:
    def __init__(self,eoa_address,transactions):
        self.eoa_address = eoa_address.lower()
        self.transactions = transactions

    def f1_total_transactions_sent(self):
        return len([x for x in self.transactions if x['from'] == self.eoa_address])

    def f2_total_transactions_received(self):
        return len([x for x in self.transactions if x['to'] == self.eoa_address])

    def f3_value_out(self):
        return len([x for x in self.transactions if x['to'] == self.eoa_address])
    
    def f3_value_out(self):
        return sum([int(x['value']) for x in self.transactions if x['from'] == self.eoa_address])
    
    def f4_value_in(self):
        return sum([int(x['value']) for x in self.transactions if x['to'] == self.eoa_address])
    
    def f5_value_difference(self):
        return abs(self.f3_value_out()-self.f4_value_in())
    
    def f6_number_of_distinct_address_contacted(self):
        seen = set()
        dupes = []
        for x in self.transactions:
            if (x['from'] == self.eoa_address and  x['to'] in seen) :
                dupes.append(x['to'])
            else:
                seen.add(x['to'])
                
            if (x['to'] == self.eoa_address and  x['from'] in seen) :
                dupes.append(x['from'])
            else:
                seen.add(x['from'])  
        return len(seen)-1
    
    def f7_total_transactions_sent_received(self):
        return self.f1_total_transactions_sent() + self.f2_total_transactions_received()
    
    def f8_total_transactions_sent_to_unique_address(self):
        all_transactions_sent = [x for x in self.transactions if x['from'] == self.eoa_address]
        seen = set()
        dupes = []
        for x in all_transactions_sent:
            if (x['to'] in seen) :
                dupes.append(x['to'])
            else:
                seen.add(x['to'])
        return len(seen)
    
    def f9_total_transactions_received_from_unique_address(self):
        all_transactions_sent = [x for x in self.transactions if x['to'] == self.eoa_address]
        seen = set()
        dupes = []
        for x in all_transactions_sent:
            if (x['from'] in seen) :
                dupes.append(x['from'])
            else:
                seen.add(x['from'])
        return len(seen)
    
    def f10_first_transaction_time(self):
        return self.transactions[0]['timeStamp']    
    
    def f11_last_transaction_time(self):
        return self.transactions[len(self.transactions)-1]['timeStamp']

    def f12_transction_active_duration(self):
        import datetime
        a = datetime.datetime.fromtimestamp(int(self.f10_first_transaction_time()))
        b = datetime.datetime.fromtimestamp(int(self.f11_last_transaction_time()))
        return (b-a).total_seconds()

In [179]:
eoa = EOA('0xbCEaA0040764009fdCFf407e82Ad1f06465fd2C4',input_dict['result'])

In [180]:
print(eoa.f1_total_transactions_sent())
print(eoa.f2_total_transactions_received())
print(eoa.f3_value_out())
print(eoa.f4_value_in())
print(eoa.f5_value_difference())
print(eoa.f6_number_of_distinct_address_contacted())
print(eoa.f7_total_transactions_sent_received())
print(eoa.f8_total_transactions_sent_to_unique_address())
print(eoa.f9_total_transactions_received_from_unique_address())
print(eoa.f10_first_transaction_time())
print(eoa.f11_last_transaction_time())
print(eoa.f12_transction_active_duration())

2
1
25533613908758401081460
25533614328758401081460
420000000000000
2
3
1
1
1533974291
1552439269
18464978.0
